## **2. Prétraitement**
- Segmentation (phrases)
- Tokenization (mots)
- Étiquetage morphosyntaxique (POS Tagging) 
- (Lemmatisation - *sur la glace ; ça pourrait permettre d'éviter les faux négatifs dûs à des variations singulier/pluriel quand on essaie d'extraire les termes qui existent déjà dans la taxonomie*)
- Filtrage (stopwords)
- Extraction de termes complexes (MWE / n-grammes / segments répétés)
- Chunking / Filtrage par patrons syntaxiques (basés sur les patrons fréquents dans les MeSH)
- Extraction de collocations significatives (en fonction du Log-likelihood ratio)
- Extraction de concordances (KWIC) pour un ensemble de mots-clés d'intérêt
- Extraction de termes MeSH présents dans les données

### **Lire le corpus** 

In [1]:
lng = 'fr'
acteur = 'pinel'
tag = ''
if tag:
    file = acteur + '/' + acteur + '_' + tag + '.csv'

else:
    if lng == 'fr':
        file = acteur +'.csv'
    if lng == 'en':
        file = acteur + '_en.csv'

In [2]:
import shutil, re, random
from os import listdir, chdir, path
from pathlib import Path
from pandas import *
import glob

import nltk
#nltk.download(['popular'])
from nltk.tokenize import RegexpTokenizer
tokenizer_re = RegexpTokenizer(r"\w\'|\w+")
from nltk import bigrams, trigrams, ngrams, everygrams
from nltk.probability import FreqDist


import treetaggerwrapper
tagger = treetaggerwrapper.TreeTagger(TAGLANG=lng)


from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.stats import binom, chi2

c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\treetaggerwrapper.py:739: FutureWarning: Possible nested set at position 8
  punct2find_re = re.compile("([^ ])([[" + ALONEMARKS + "])",
c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\treetaggerwrapper.py:2043: FutureWarning: Possible nested set at position 152
  DnsHostMatch_re = re.compile("(" + DnsHost_expression + ")",
c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\treetaggerwrapper.py:2067: FutureWarning: Possible nested set at position 409
  UrlMatch_re = re.compile(UrlMatch_expression, re.VERBOSE | re.IGNORECASE)
c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\treetaggerwrapper.py:2079: FutureWarning: Possible nested set at position 192
  EmailMatch_re = re.compile(EmailMatch_expression, re.VERBOSE | re.IGNORECASE)


In [3]:
def lire_corpus(acteur = acteur, langue=lng):
    base_path = '../03-corpus/2-data/'
    
    folder_path = path.join(base_path, '1-' + langue, acteur)
    all_files = glob.glob(path.join(folder_path, "*.csv"))
    tags = [f.split('_')[1][:-4] for f in listdir(folder_path)]

    df = DataFrame()
    for f, tag in zip(all_files, tags):
        csv = read_csv(f, encoding='utf-8', sep=',')
        csv = csv[~csv["Address"].str.contains('pdf')] #  Problèmes 
        #csv = csv['text']
        # Using DataFrame.insert() to add a column
        df = concat([df, csv]) [['Corpus', 'Sous-corpus', 'Title', 'text']]
        #df = concat([df, csv]) [['Corpus', 'Sous-corpus', 'Title', 'Type', 'text']]
    return df

In [4]:
data = lire_corpus(acteur) 
nb_docs = len(data)
print("On a un corpus de {} documents.".format(nb_docs))

On a un corpus de 115 documents.


In [5]:
base_path = '../03-corpus/2-data/'
folder_path = path.join(base_path, '1-' + lng, acteur)

data.to_csv(folder_path + '.csv')
data

,Corpus,Sous-corpus,Title,text
0,pinel,Administration,Horaire du parloir - Institut national de psyc...,Horaire du parloir OUVERT OUVERT – RÉSERVATION...
1,pinel,Administration,<span>Horaire et réservation<//span>,Horaire du parloir OUVERT OUVERT – RÉSERVATION...
2,pinel,Administration,L’Institut national de psychiatrie légale Phil...,Retour aux nouvelles Date de publication 5 fév...
0,pinel,Carrières,Agent(e) administratif(ive) classe 2 - Liste d...,Retour aux carrières Date de publication 5 aoû...
1,pinel,Carrières,Agente administrative classe 1 – Unité F-1 - I...,Retour aux carrières Date de publication 5 aoû...
...,...,...,...,...
11,pinel,Soins et services,Philosophie des soins - Institut national de p...,Philosophie des soins Notre philosophie des so...
12,pinel,Soins et services,Soins et services - Institut national de psych...,Soins et services Le patient est au cœur de la...
13,pinel,Soins et services,<span>Soins et services<//span>,Soins et services Le patient est au cœur de la...
14,pinel,Soins et services,Unité de soins - Institut national de psychiat...,Unité de soins Un milieu de vie et de soins hu...


### **Nettoyage**

In [6]:
punct = '[!#$%&•►*+,;\/\\<=>?@[\]^_{|}~©«»—“”–—]'
spaces = '\s+'
postals = '([a-zA-Z]+\d+|\d+[a-zA-Z]+)+'
# phones = '\d{3}\s\d{3}-\d{4}' #très simple (trop)

text = [str(t).strip('\n').lower().replace('’', '\'') for t in data['text'].tolist()]
text = [re.sub(spaces, ' ', t) for t in text]
text = [re.sub(postals, ' STOP ', t) for t in text]
text = [re.sub(punct, ' STOP ', t) for t in text]
text = [t.replace("  ", " " ) for t in text]

### **Extraire un échantillon aléatoire**

Sinon, on n'arrive pas à traiter la totalité du corpus pour des raisons de performance

In [7]:
def sample_corpus(corpus, ratio):
    n = round(ratio * len(corpus))
    corpus = random.sample(text, n)
    print("On va travailler sur un échantillon correspondant à environ " + str(ratio * 100) + " % des documents du corpus, soit {} documents". format(len(corpus)))
    return " ".join(corpus)
    
corpus = sample_corpus(text, 1)

On va travailler sur un échantillon correspondant à environ 100 % des documents du corpus, soit 115 documents


### **Filtrage (MWE - stopwords formés de plusieurs tokens)**
Surtout pour filtrer les expressions relatives à l'architecture d'information / navigation Web

In [8]:
def filter_mwesw(corpus):
    file_mwesw = '../04-filtrage/mwe_stopwords.txt'
    with open (file_mwesw, 'r', encoding='utf-8') as f:
        mwe_sw = [t.lower().strip('\n') for t in f.readlines()]
    for mwe in mwe_sw:
        corpus = corpus.replace(mwe, ' STOP ').replace('  ', " ")
    return corpus

In [9]:
corpus = filter_mwesw(corpus)

### **Tokenisation / POS tagging** (TreeTagger)  
https://github.com/miotto/treetagger-python/blob/master/README.rst  
https://treetaggerwrapper.readthedocs.io/en/latest/

In [10]:
# Ici, on tokenise une première fois avec le Regex Tokenizer de NLTK pour voir combien de temps ça devrait 
# prendre au Tree Tagger pour tokeniser et tagger notre corpus
def tok(corpus):
    # Seulement les caractères alphabétiques
    tokens = tokenizer_re.tokenize(corpus)
    print("Avec le RegExpTokenizer, notre corpus contient {} tokens.".format(len(tokens)))
    temps = round(len(tokens) / 15000 / 60)
    print('Le POS tagging devrait prendre environ {} minutes.'.format(temps))

temps = tok(corpus)

Avec le RegExpTokenizer, notre corpus contient 78246 tokens.
Le POS tagging devrait prendre environ 0 minutes.


In [11]:
def tagging(corpus):
    output = []
    for t in tagger.tag_text(corpus):
        try: 
            output.append([t.split('\t')[0], t.split('\t')[1]])
        except Exception as e:
            output.append(('STOP', 'NAM'))

    return output


In [12]:
tagged = tagging(corpus)
tokens = [t[0] for t in tagged]

In [15]:
tagged

[['STOP', 'INT'],
 ['le', 'DET:ART'],
 ['patient', 'ADJ'],
 ['est', 'VER:pres'],
 ['au', 'PRP:det'],
 ['cœur', 'NOM'],
 ['de', 'PRP'],
 ['la', 'DET:ART'],
 ['mission', 'NOM'],
 ['de', 'PRP'],
 ["l'", 'DET:ART'],
 ['institut', 'NOM'],
 ['.', 'SENT'],
 ['les', 'DET:ART'],
 ['personnes', 'NOM'],
 ['qui', 'PRO:REL'],
 ['sont', 'VER:pres'],
 ['admises', 'VER:pper'],
 ['reçoivent', 'VER:pres'],
 ['les', 'DET:ART'],
 ['soins', 'NOM'],
 ['et', 'KON'],
 ['les', 'DET:ART'],
 ['services', 'NOM'],
 ['appropriés', 'ADJ'],
 ['.', 'SENT'],
 ['des', 'PRP:det'],
 ['équipes', 'NOM'],
 ['multidisciplinaires', 'ADJ'],
 ['regroupant', 'VER:ppre'],
 ['différents', 'ADJ'],
 ['professionnels', 'NOM'],
 ['de', 'PRP'],
 ['la', 'DET:ART'],
 ['santé', 'NOM'],
 ['travaillent', 'VER:pres'],
 ['ensemble', 'ADV'],
 ['pour', 'PRP'],
 ['assurer', 'VER:infi'],
 ['le', 'DET:ART'],
 ['mieux-être', 'NOM'],
 ['de', 'PRP'],
 ['la', 'DET:ART'],
 ['personne', 'NOM'],
 ['dans', 'PRP'],
 ['le', 'DET:ART'],
 ['cadre', 'NOM'],
 ['

### **Lemmatisation** (FrenchLefffLemmatizer)
Sur pause ; peut-être qu'il vaudrait mieux le faire plus tard (au moment d'identifier si chaque terme existe déjà dans la taxo ou non) afin d'éviter de complexifier toutes les autres étapes

In [135]:
# from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
# lemmatizer = FrenchLefffLemmatizer()

#### **Mapping POS Tags** (FRMG)
On doit mapper les étiquettes morphosyntaxiques du TreeTagger à celles que prend le lemmatiseur (celles issues de FRMG)
#http://alpage.inria.fr/frmgwiki/content/tagset-frmg

In [136]:
# file_path = '../04-filtrage/mapping_treeTagger_lefff.csv'

# with open(file_path) as f:
#    csv = read_csv(f)

# treeTag = [term for term in csv['TreeTagger'].tolist()] 
# lefff = [term for term in csv['Lefff'].tolist()]

# mapping = {term : lefff[treeTag.index(term)] for term in treeTag}

In [137]:
# df_tagged = DataFrame(tagged, columns=['Token', 'Tag TreeTagger'])
# df_tagged['Tag Lefff'] = df_tagged['Tag TreeTagger'].map(mapping)
# df_tagged['Lemme'] = df_tagged['Token'] # S'il est pas capable de lemmatiser ensuite, on garde l'expression telle quelle

In [138]:
# tagged_dict = df_tagged.to_dict('records')

In [139]:
# # tokens = df_tagged['Token'].tolist()
# # tags = df_tagged['Tag Lefff'].tolist()
# # tagged_lefff = [(t, tg) for t,tg in zip(tokens, tags)]

# for term in tagged_dict:
#     if lemmatizer.lemmatize(word=term['Token'], pos=term['Tag Lefff']) == []:
#         term_l = lemmatizer.lemmatize(word=term['Token'])
    
#     elif type(lemmatizer.lemmatize(word=term['Token'], pos=term['Tag Lefff'])) == str:
#         term_l  = lemmatizer.lemmatize(word=term['Token'], pos=term['Tag Lefff'])

#     else:
#         term_l = lemmatizer.lemmatize(word=term['Token'], pos=term['Tag Lefff'])[0][0]

#     term['Lemme'] = term_l

In [140]:
# tagged_dict

In [141]:
# tagged = DataFrame(tagged_dict)
# tagged = tagged.drop(columns=['Tag Lefff']).values.tolist()

# tagged

### **Collocations / Phrases / N-Grammes (MWE)**
https://www.kaggle.com/code/alvations/n-gram-language-model-with-nltk/notebook  

In [16]:
def extr_ngrams(tagged):
    ngrammes= list(everygrams(tagged, min_len=1, max_len=10))
    print("Avant filtrage, on a {} ngrammes.".format(len(ngrammes)))
    return ngrammes

In [17]:
ngrammes = extr_ngrams(tagged)
#ngrammes_lem = extr_ngrams(tagged_lem)

Avant filtrage, on a 810985 ngrammes.


In [18]:
ngrammes

[(['STOP', 'INT'],),
 (['STOP', 'INT'], ['le', 'DET:ART']),
 (['STOP', 'INT'], ['le', 'DET:ART'], ['patient', 'ADJ']),
 (['STOP', 'INT'], ['le', 'DET:ART'], ['patient', 'ADJ'], ['est', 'VER:pres']),
 (['STOP', 'INT'],
  ['le', 'DET:ART'],
  ['patient', 'ADJ'],
  ['est', 'VER:pres'],
  ['au', 'PRP:det']),
 (['STOP', 'INT'],
  ['le', 'DET:ART'],
  ['patient', 'ADJ'],
  ['est', 'VER:pres'],
  ['au', 'PRP:det'],
  ['cœur', 'NOM']),
 (['STOP', 'INT'],
  ['le', 'DET:ART'],
  ['patient', 'ADJ'],
  ['est', 'VER:pres'],
  ['au', 'PRP:det'],
  ['cœur', 'NOM'],
  ['de', 'PRP']),
 (['STOP', 'INT'],
  ['le', 'DET:ART'],
  ['patient', 'ADJ'],
  ['est', 'VER:pres'],
  ['au', 'PRP:det'],
  ['cœur', 'NOM'],
  ['de', 'PRP'],
  ['la', 'DET:ART']),
 (['STOP', 'INT'],
  ['le', 'DET:ART'],
  ['patient', 'ADJ'],
  ['est', 'VER:pres'],
  ['au', 'PRP:det'],
  ['cœur', 'NOM'],
  ['de', 'PRP'],
  ['la', 'DET:ART'],
  ['mission', 'NOM']),
 (['STOP', 'INT'],
  ['le', 'DET:ART'],
  ['patient', 'ADJ'],
  ['est', 'VE

### **Extraction des patrons syntaxiques**

In [19]:
def extract_patterns(ngrammes):
    patterns = []
    for ng in ngrammes:
        phrase = tuple([t[0] for t in ng])
        pattern = [t[1] for t in ng]
        patterns.append([phrase, pattern])
    return patterns

In [20]:
phrases = extract_patterns(ngrammes)
# phrases_lem = extract_patterns(ngrammes_lem)

In [21]:
# def freq(phrases):
#     return FreqDist([" ".join(t[0]).replace("' ", "'") for t in phrases])

# def freq(phrases):
#     return FreqDist([t[0] for t in phrases])

In [22]:
frequencies = FreqDist(everygrams(tokens, min_len=1, max_len=10))

In [23]:
frequencies

FreqDist({('de',): 5367, ('STOP',): 4692, ('et',): 2692, ("l'",): 2075, ('.',): 1905, ('des',): 1780, ('les',): 1754, ('la',): 1683, ("d'",): 1617, ('en',): 1574, ...})

### **Filtrage** 
On retire les n-grammes qui débutent ou se terminent par un stopword (antidictionnaire)

In [24]:
# Importer l'antidictionnaire pour filtrer les données

# # Stopwords lemmatisés
# file_path = '../04-filtrage/stopwords_lemmatized.txt'
# with open(file_path, 'r', encoding="utf-8") as f:
#     stopwords_lemmatized = [w.strip('\n').lower() for w in f.readlines()]

# Stopwords fréquents en français (non lemmatisés)
file_path = "../04-filtrage/stopwords.txt"
with open(file_path, 'r', encoding="utf-8") as f:
    stopwords = [t.lower().strip('\n') for t in f.readlines()]


# Stopwords fréquents en anglais (non lemmatisés)
file_path = '../04-filtrage/stop_words_english.txt'
with open(file_path, 'r', encoding="utf-8") as f:
    stopwords += [t.lower().strip('\n') for t in f.readlines()]

*LONG*

In [25]:
def filtrer_stopwords(x): # On s'assure aussi que le premier terme du ngramme est un NOM pour avoir des syntagmes nominaux
    if lng == 'en':
        nom = 'NN'
    if lng == 'fr':
        nom = 'NOM'
    return [term for term in x if not 'STOP' in term[0] and not term[0][0] in stopwords and not term[0][-1] in stopwords \
        and not 'NUM' in term[1] and term[1][0] == nom and not '.' in term[0] and not '-' in term[0] and not ':' in term[0]\
        
        # Une parenthèse fermante peut juste se trouver comme dernier token
        # Si une parenthèse est ouverte, elle doit aussi être fermée (et vice versa)
        and not ')' in term[0][:-1] and not ('(' in term[0] and not ')' in term[0]) \
        and not (')' in term[0] and not '(' in term[0])]

In [26]:
phrases = filtrer_stopwords(phrases)

On retire les n-grammes qui débutent ou se terminent par un token dont la longueur est inférieure à 2 caractères ou supérieure à 18 caractères

In [27]:
def filter_len(x):
    return [term for term in x if \
        (len(term[0][0]) > 2 or term[0][0] == '(')  and (len(term[0][-1]) > 2 or term[0][-1] == ')') and \
        len(term[0][0]) < 18 and len(term[0][-1]) < 18]

In [28]:
phrases = filter_len(phrases)

In [29]:
phrases

[[('cœur',), ['NOM']],
 [('cœur', 'de', 'la', 'mission', 'de', "l'", 'institut'),
  ['NOM', 'PRP', 'DET:ART', 'NOM', 'PRP', 'DET:ART', 'NOM']],
 [('institut',), ['NOM']],
 [('soins',), ['NOM']],
 [('soins', 'et', 'les', 'services'), ['NOM', 'KON', 'DET:ART', 'NOM']],
 [('soins', 'et', 'les', 'services', 'appropriés'),
  ['NOM', 'KON', 'DET:ART', 'NOM', 'ADJ']],
 [('services',), ['NOM']],
 [('services', 'appropriés'), ['NOM', 'ADJ']],
 [('équipes',), ['NOM']],
 [('équipes', 'multidisciplinaires', 'regroupant'),
  ['NOM', 'ADJ', 'VER:ppre']],
 [('équipes',
   'multidisciplinaires',
   'regroupant',
   'différents',
   'professionnels'),
  ['NOM', 'ADJ', 'VER:ppre', 'ADJ', 'NOM']],
 [('équipes',
   'multidisciplinaires',
   'regroupant',
   'différents',
   'professionnels',
   'de',
   'la',
   'santé'),
  ['NOM', 'ADJ', 'VER:ppre', 'ADJ', 'NOM', 'PRP', 'DET:ART', 'NOM']],
 [('équipes',
   'multidisciplinaires',
   'regroupant',
   'différents',
   'professionnels',
   'de',
   'la',
   

On retire les n-grammes qui apparaissent moins de 15 fois dans le corpus

In [30]:
def filter_freq(x):
    return [term for term in x if frequencies[tuple(term[0])] > 15]

In [31]:
phrases = filter_freq(phrases)

In [32]:
phrases

[[('institut',), ['NOM']],
 [('soins',), ['NOM']],
 [('services',), ['NOM']],
 [('équipes',), ['NOM']],
 [('professionnels',), ['NOM']],
 [('santé',), ['NOM']],
 [('cadre',), ['NOM']],
 [('programmes',), ['NOM']],
 [('respect',), ['NOM']],
 [('maladie',), ['NOM']],
 [('maintien',), ['NOM']],
 [('soins',), ['NOM']],
 [('services',), ['NOM']],
 [('rétablissement',), ['NOM']],
 [('reconnaissance',), ['NOM']],
 [('soutien',), ['NOM']],
 [('patient',), ['NOM']],
 [('planification',), ['NOM']],
 [('organisation',), ['NOM']],
 [('services',), ['NOM']],
 [('patients',), ['NOM']],
 [('accès',), ['NOM']],
 [('soins',), ['NOM']],
 [('services',), ['NOM']],
 [('santé',), ['NOM']],
 [('mandat',), ['NOM']],
 [('mandat',), ['NOM']],
 [('services',), ['NOM']],
 [('développement',), ['NOM']],
 [('ensemble',), ['NOM']],
 [('patients',), ['NOM']],
 [('institut',), ['NOM']],
 [('programmation',), ['NOM']],
 [('clientèle',), ['NOM']],
 [('soutien',), ['NOM']],
 [('intégration',), ['NOM']],
 [('soutien',), 

In [33]:
#phrases = [[" ".join(term[0]).replace("' ", "'"), " ".join(term[1])] for term in phrases]
phrases = [[term[0], " ".join(term[1])] for term in phrases]
# phrases_lemmatized = filtrer_phrases(phrases_lemmatized, freq_lemmatized)

In [34]:
for phrase in phrases:
    phrase.append(frequencies[tuple(phrase[0])])

In [35]:
print("Après filtrage, on a {} occurrences de ngrammes.".format(len(phrases))) 

Après filtrage, on a 29954 occurrences de ngrammes.


In [36]:
phrases[:15]

[[('institut',), 'NOM', 568],
 [('soins',), 'NOM', 211],
 [('services',), 'NOM', 193],
 [('équipes',), 'NOM', 35],
 [('professionnels',), 'NOM', 104],
 [('santé',), 'NOM', 212],
 [('cadre',), 'NOM', 54],
 [('programmes',), 'NOM', 71],
 [('respect',), 'NOM', 31],
 [('maladie',), 'NOM', 53],
 [('maintien',), 'NOM', 24],
 [('soins',), 'NOM', 211],
 [('services',), 'NOM', 193],
 [('rétablissement',), 'NOM', 27],
 [('reconnaissance',), 'NOM', 19]]

In [37]:
DataFrame(phrases, columns=["Expression", "Patron syntaxique", "Fréquence"])

phrases

[[('institut',), 'NOM', 568],
 [('soins',), 'NOM', 211],
 [('services',), 'NOM', 193],
 [('équipes',), 'NOM', 35],
 [('professionnels',), 'NOM', 104],
 [('santé',), 'NOM', 212],
 [('cadre',), 'NOM', 54],
 [('programmes',), 'NOM', 71],
 [('respect',), 'NOM', 31],
 [('maladie',), 'NOM', 53],
 [('maintien',), 'NOM', 24],
 [('soins',), 'NOM', 211],
 [('services',), 'NOM', 193],
 [('rétablissement',), 'NOM', 27],
 [('reconnaissance',), 'NOM', 19],
 [('soutien',), 'NOM', 35],
 [('patient',), 'NOM', 25],
 [('planification',), 'NOM', 43],
 [('organisation',), 'NOM', 43],
 [('services',), 'NOM', 193],
 [('patients',), 'NOM', 114],
 [('accès',), 'NOM', 159],
 [('soins',), 'NOM', 211],
 [('services',), 'NOM', 193],
 [('santé',), 'NOM', 212],
 [('mandat',), 'NOM', 23],
 [('mandat',), 'NOM', 23],
 [('services',), 'NOM', 193],
 [('développement',), 'NOM', 156],
 [('ensemble',), 'NOM', 29],
 [('patients',), 'NOM', 114],
 [('institut',), 'NOM', 568],
 [('programmation',), 'NOM', 19],
 [('clientèle',),

In [38]:
print("Après filtrage, on a {} ngrammes uniques.".format(len(DataFrame(phrases).drop_duplicates())))

Après filtrage, on a 564 ngrammes uniques.


### **Filtrage (Patrons syntaxiques)**  
Lossio-Ventura, J. A., Jonquet, C., Roche, M., & Teisseire, M. (2014). Biomedical Terminology Extraction : A new combination of Statistical and Web Mining Approaches. 421. https://hal-lirmm.ccsd.cnrs.fr/lirmm-01056598

On veut aller extraire les structures syntaxiques les plus courantes dans les MeSH pour filtrer notre corpus selon celles-ci (inspiré de la méthodologie de l'article ci-dessus ; voir le Notebook *Mesh_extract.ipynb*). Pour ce faire, nous allons donc ne sélectionner que les ngrammes qui y correspondent. 

In [39]:
file_patterns = '../04-filtrage/MeSH/mesh_patterns-fr.csv'

with open (file_patterns, 'r') as f:
    patterns = read_csv(f)
    patterns = patterns['Structure'].tolist() #[:200] # Pour prendre les 200 structures syntaxiques les plus fréquentes dans les MeSH

In [40]:
def filter_patterns(phrases):
    return [t for t in phrases if t[1] in patterns and not 'NUM' in t[1]] # and not 'NOM NOM' in t[1]
# terms_lemmatized = [t for t in phrases_lemmatized if t[1] in patterns]

In [41]:
terms = filter_patterns(phrases)
#terms = phrases

In [42]:
terms

[[('institut',), 'NOM', 568],
 [('soins',), 'NOM', 211],
 [('services',), 'NOM', 193],
 [('équipes',), 'NOM', 35],
 [('professionnels',), 'NOM', 104],
 [('santé',), 'NOM', 212],
 [('cadre',), 'NOM', 54],
 [('programmes',), 'NOM', 71],
 [('respect',), 'NOM', 31],
 [('maladie',), 'NOM', 53],
 [('maintien',), 'NOM', 24],
 [('soins',), 'NOM', 211],
 [('services',), 'NOM', 193],
 [('rétablissement',), 'NOM', 27],
 [('reconnaissance',), 'NOM', 19],
 [('soutien',), 'NOM', 35],
 [('patient',), 'NOM', 25],
 [('planification',), 'NOM', 43],
 [('organisation',), 'NOM', 43],
 [('services',), 'NOM', 193],
 [('patients',), 'NOM', 114],
 [('accès',), 'NOM', 159],
 [('soins',), 'NOM', 211],
 [('services',), 'NOM', 193],
 [('santé',), 'NOM', 212],
 [('mandat',), 'NOM', 23],
 [('mandat',), 'NOM', 23],
 [('services',), 'NOM', 193],
 [('développement',), 'NOM', 156],
 [('ensemble',), 'NOM', 29],
 [('patients',), 'NOM', 114],
 [('institut',), 'NOM', 568],
 [('programmation',), 'NOM', 19],
 [('clientèle',),

In [43]:
print("Le filtrage syntaxique élimine environ {} % des termes".format(round((len(phrases) - len(terms)) / len(phrases) * 100)))
print("On avait {} ngrammes, ".format(len(phrases)) + "on en a maintenant {}.".format(len(terms)))

Le filtrage syntaxique élimine environ 21 % des termes
On avait 29954 ngrammes, on en a maintenant 23536.


In [44]:
# import pandas as pd

# def extract_terms(liste_terms):
#     file_path = '../04-filtrage/output/'
#     tab = pd.DataFrame(terms, columns= ["Expression", "Structure syntaxique", "Fréquence"]).drop_duplicates(subset='Expression', keep="last")
#     tab.sort_values(["Fréquence"], 
#                         axis=0,
#                         ascending=[False], 
#                         inplace=True)

#     return 

#     # file_path = path.join(file_path, tag, tag)                    
#     # tab.to_csv(file_path + '_terms.csv')

# extract_terms(terms)
# #extract_terms(terms_lemmatized)

In [45]:
for phrase in terms:
    phrase[0] = tuple(phrase[0])


terms_patterns = DataFrame(terms, columns = ["Expression", "Structure syntaxique", "Fréquence"])
terms_patterns = terms_patterns.to_dict('records')
dict_patterns = {}
for term in terms_patterns:
     exp = term['Expression']
     pattern = term['Structure syntaxique']
     dict_patterns[exp] = pattern

In [46]:
terms_patterns

[{'Expression': ('institut',),
  'Structure syntaxique': 'NOM',
  'Fréquence': 568},
 {'Expression': ('soins',), 'Structure syntaxique': 'NOM', 'Fréquence': 211},
 {'Expression': ('services',),
  'Structure syntaxique': 'NOM',
  'Fréquence': 193},
 {'Expression': ('équipes',), 'Structure syntaxique': 'NOM', 'Fréquence': 35},
 {'Expression': ('professionnels',),
  'Structure syntaxique': 'NOM',
  'Fréquence': 104},
 {'Expression': ('santé',), 'Structure syntaxique': 'NOM', 'Fréquence': 212},
 {'Expression': ('cadre',), 'Structure syntaxique': 'NOM', 'Fréquence': 54},
 {'Expression': ('programmes',),
  'Structure syntaxique': 'NOM',
  'Fréquence': 71},
 {'Expression': ('respect',), 'Structure syntaxique': 'NOM', 'Fréquence': 31},
 {'Expression': ('maladie',), 'Structure syntaxique': 'NOM', 'Fréquence': 53},
 {'Expression': ('maintien',), 'Structure syntaxique': 'NOM', 'Fréquence': 24},
 {'Expression': ('soins',), 'Structure syntaxique': 'NOM', 'Fréquence': 211},
 {'Expression': ('service

### **Filtrage (Collocations statistiquement significatives)** Log-Likelihood Ratio

[Notebook - Collocation extraction methodologies compared](https://notebooks.githubusercontent.com/view/ipynb?azure_maps_enabled=false&browser=chrome&color_mode=auto&commit=33868e847376764d7733cd958986c88dedfaec97&device=unknown&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f746f64642d636f6f6b2f4d4c2d596f752d43616e2d5573652f333338363865383437333736373634643737333363643935383938366338386465646661656339372f70726f626162696c69737469635f6c616e67756167655f6d6f64656c696e672f636f6c6c6f636174696f6e5f65787472616374696f6e732e6970796e62&enterprise_enabled=false&logged_in=false&nwo=todd-cook%2FML-You-Can-Use&path=probabilistic_language_modeling%2Fcollocation_extractions.ipynb&platform=android&repository_id=167140788&repository_type=Repository&version=102)

On applique un test d'hypothèse statistique aux n-grammes sur lesquels une probabilité a été mesurée (Log-likelihood ratio) - seuls les n-grammes dont le test est significatif seront conservés.
On considère que l'apparition de ces collocations dans notre corpus n'est pas dûe au hasard.

In [47]:
def loglikelihood_ratio(c_prior, c_n, c_ngram, N):
    """
    Compute the ratio of two hypotheses of likelihood and return the ratio.
    The formula here and test verification values are taken from 
    Manning & Schūtze _Foundations of Statistical Natural Language Processing_ p.172-175
    Parameters:
    c_prior: count of word 1 if bigrams or count of [w1w2 .. w(n-1)] if ngram
    c_n : count of word 2 if bigrams or count of wn if ngram
    c12: count of bigram (w1, w2) if bigram or count of ngram if ngram
    N: the number of words in the corpus
    """

    p = c_n / N
    p1 = c_ngram / c_prior
    p2 = (c_n - c_ngram) / (N - c_prior)   
    # We proactively trap a runtimeWarning: divide by zero encountered in log,
    # which may occur with extreme collocations
    import warnings
    with warnings.catch_warnings(): # this will reset our filterwarnings setting
        warnings.filterwarnings('error')
        try:
            return (np.log(binom.pmf(c_ngram, c_prior, p)) 
                    + np.log(binom.pmf(c_n - c_ngram, N - c_prior, p)) 
                    - np.log(binom.pmf(c_ngram, c_prior, p1) )
                    - np.log(binom.pmf(c_n - c_ngram, N - c_prior, p2)))             
        except Warning:
            return np.inf 

In [48]:
len_prior = len(terms)
print("Au départ, on a {} ngrammes.".format(len_prior))

Au départ, on a 23536 ngrammes.


In [49]:
frequencies

FreqDist({('de',): 5367, ('STOP',): 4692, ('et',): 2692, ("l'",): 2075, ('.',): 1905, ('des',): 1780, ('les',): 1754, ('la',): 1683, ("d'",): 1617, ('en',): 1574, ...})

In [50]:
# Pour le calcul des probabilités, on a besoin de traiter séparément les ngrammes selon la valeur de n
N = len(tokens)
fd_tokens = nltk.FreqDist(tokens)

In [51]:
def llr_ngrammes(terms):
    llr = []

    ngrammes = [term[0] for term in terms]
        
    for t in ngrammes:
        if len(t) == 1:
            try:
                llr.append({'Terme' : str(t[0]), 'Structure syntaxique': dict_patterns[t], 'Fréquence (TF)' : fd_tokens[str(t[0])], 'LLR': '-', 'p-value': '-'})
            except Exception as e:
                print(t, str(e))
        else:
            c_prior = frequencies[t[:-1]] # Antécédent = P(w1w2..w_n-1) (si on considère que P(w1w2...wn) = P(wn) | P(w1w2...w_n-1)
            c_n = fd_tokens[t[-1]]     # Dernier mot du ngramme  P(wn)
            c_ngram = frequencies[t]             # Le ngramme lui-même P(w1w2w3..wn)

            try:
                res = -2 * loglikelihood_ratio(c_prior, c_n, c_ngram, N)
                p_value = chi2.sf(res, 1) # 1 degrees of freedom
                #if res == float('-inf') :
                #    res = 50000

                if p_value < 0.01 or (res == float('-inf')):
                    #llr.append({'Collocation' : " ".join(t).replace("' ", "'").replace("( ", "(").replace(" )", ")"), 'Structure syntaxique': dict_patterns[" ".join(t).replace("' ", "'")], 'Fréquence' : c_ngram, 'LLR': res, 'p-value': p_value})
                    llr.append({'Terme' : t, 'Structure syntaxique': dict_patterns[t], 'Fréquence (TF)' : c_ngram, 'LLR': res, 'p-value': p_value})
            
            except Exception as e:
                print(t, str(e))
            
    return llr


In [52]:
significant_coll = llr_ngrammes(terms)
#terms = [{'Collocation' : t[0], 'Structure syntaxique': t[1], 'Fréquence' : t[2]} for t in terms] # Si on veut pas appliquer le LLR

In [53]:
significant_coll


[{'Terme': 'institut',
  'Structure syntaxique': 'NOM',
  'Fréquence (TF)': 568,
  'LLR': '-',
  'p-value': '-'},
 {'Terme': 'soins',
  'Structure syntaxique': 'NOM',
  'Fréquence (TF)': 211,
  'LLR': '-',
  'p-value': '-'},
 {'Terme': 'services',
  'Structure syntaxique': 'NOM',
  'Fréquence (TF)': 193,
  'LLR': '-',
  'p-value': '-'},
 {'Terme': 'équipes',
  'Structure syntaxique': 'NOM',
  'Fréquence (TF)': 35,
  'LLR': '-',
  'p-value': '-'},
 {'Terme': 'professionnels',
  'Structure syntaxique': 'NOM',
  'Fréquence (TF)': 104,
  'LLR': '-',
  'p-value': '-'},
 {'Terme': 'santé',
  'Structure syntaxique': 'NOM',
  'Fréquence (TF)': 212,
  'LLR': '-',
  'p-value': '-'},
 {'Terme': 'cadre',
  'Structure syntaxique': 'NOM',
  'Fréquence (TF)': 54,
  'LLR': '-',
  'p-value': '-'},
 {'Terme': 'programmes',
  'Structure syntaxique': 'NOM',
  'Fréquence (TF)': 71,
  'LLR': '-',
  'p-value': '-'},
 {'Terme': 'respect',
  'Structure syntaxique': 'NOM',
  'Fréquence (TF)': 31,
  'LLR': '-',


In [54]:
print('Après avoir calculé le log-likelihood ratio, on a retiré {} collocations qui n\'étaient pas statistiquement significatives.'.format(len(terms) - len(significant_coll)))
print('Ça représente environ {} % de nos n-grammes.'.format(round((len(terms) - len(significant_coll)) / len(terms) *100 )))

Après avoir calculé le log-likelihood ratio, on a retiré 0 collocations qui n'étaient pas statistiquement significatives.
Ça représente environ 0 % de nos n-grammes.


In [55]:
df = DataFrame(significant_coll).sort_values(by = "Fréquence (TF)", ascending=False).drop_duplicates()

# On veut faire join pour tous les termes[collocation]
def join_term(x):
    if type(x) == tuple:
        return " ".join(x).replace("' ", "'").replace("( ", "(").replace(" )", ")")
    else:
        return x

df['Terme'] = df['Terme'].apply(lambda x: join_term(x))

if lng == 'en':
    acteur = acteur + '_' + lng
if tag:
    output_path = path.join('../04-filtrage/output/', acteur + '_' + tag + '_significant-collocations.csv') 
else:
    output_path = path.join('../04-filtrage/output/', acteur + '_significant-collocations.csv') 

df.to_csv(output_path)

df

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value
0,institut,NOM,568,-,-
15051,psychiatrie,NOM,476,-,-
11872,psychiatrie légale,NOM ADJ,416,-inf,1.0
17782,candidature,NOM,285,-,-
7381,philippe-pinel,NOM,233,-,-
...,...,...,...,...,...
13662,ordre des psychologues du québec,NOM PRP:det NOM PRP:det NOM,16,253.257371,0.0
13664,psychologues du québec,NOM PRP:det NOM,16,253.257371,0.0
10547,institut philippe-pinel,NOM ADJ,16,45.598079,0.0
14877,études secondaires,NOM ADJ,16,250.138932,0.0


### **Filtrage - Fréquence documentaire**
** Il y aurait quelque chose à modifier ici ; on a une fréquence documentaire de 0 pour les ngrammes qui n'ont plus la forme exacte qu'ils avaient dans le corpus orginal. **

In [56]:
dfs = {term: len([doc for doc in text if term in doc]) for term in df['Terme'].tolist()}

In [57]:
max_df = round(0.98 * nb_docs)        # Pour rejeter les termes qui se retrouvent dans plus de 98% des documents 
max_df

113

In [58]:
max_df_observed = dfs[max(dfs, key=dfs.get)] # Voir quelle est la fréquence documentaire maximale qu'on retrouve
max_df_observed

108

In [59]:
max_df_to_remove = {term:df for term,df in dfs.items() if df > max_df}
max_df_to_remove

{}

In [60]:
dfs = {term:df for term,df in dfs.items() if df < max_df and df > 0} # Si df = 0, c'est un artefact créé par le prétraitement lui-même

Porter attention aux termes qui sortent ici - en comptant la fréquence documentaire, techniquement on ne devrait pas retrouver de 0

In [61]:
zeros =  {term:df for term,df in dfs.items() if df == 0}  
print(len(zeros))
zeros 

0


{}

In [62]:
dfs = DataFrame(list(dfs.items()),columns = ['Terme','Fréquence documentaire (DF)']) 

dfs

,Terme,Fréquence documentaire (DF)
0,institut,108
1,psychiatrie,99
2,psychiatrie légale,94
3,candidature,71
4,philippe-pinel,94
...,...,...
432,ordre des psychologues du québec,11
433,psychologues du québec,11
434,institut philippe-pinel,5
435,études secondaires,16


In [63]:
df = df.merge(dfs, on='Terme').drop_duplicates()

df.sort_values(['Fréquence (TF)'], 
            axis=0,
            ascending=[False], 
            inplace=True)

In [64]:
df

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF)
0,institut,NOM,568,-,-,108
1,psychiatrie,NOM,476,-,-,99
2,psychiatrie légale,NOM ADJ,416,-inf,1.0,94
3,candidature,NOM,285,-,-,71
4,philippe-pinel,NOM,233,-,-,94
...,...,...,...,...,...,...
417,programmes de traitement,NOM PRP NOM,16,200.173653,0.0,5
416,statut liste de rappel,NOM NOM PRP NOM,16,262.04235,0.0,10
415,temps complet permanent titre du poste,NOM ADJ ADJ NOM PRP:det NOM,16,190.608353,0.0,2
414,dotation,NOM,16,-,-,6


In [65]:
list_terms = df['Terme'].tolist()

### **KWIC (Keyword in Context)**
Termes d'intérêt : 
- « Programme »
- « Plan »
- « Service(s) de » 
- « Intervenant(e) en »
- « Professionnel de »
- « Institut (du/de) »
- « Groupe de recherche en »
- « Personne »
- « Infirmière (en) »

In [66]:
# Dans notre cas on veut que ça débute par le mot-clé donc le contexte est un peu plus simple
# penser à généraliser avec des expressions régulières
# kw = ['programme', 'plan ', 'service', 'intervenant', 'infirmière en', 'institut', 'groupe de recherche', 'personne', 'maladie']

# ngrammes_kwic = [" ".join([t[0] for t in ng]).replace("' ", "'").replace("( ", "(").replace(" )", ")") for ng in ngrammes]

In [67]:
# ngrammes_kwic = [t for t in ngrammes_kwic if not 'STOP' in t]

In [68]:
# extrant = DataFrame(columns=['Mot-clé','Concordance', 'Fréquence (TF)'])
# kwic = {w : [] for w in kw} 

In [69]:
# for t in ngrammes_kwic: # on pourrait aussi chercher dans les terms, mais on perd certains termes d'intérêt avec le filtrage syntaxique
#     for w in kw:
#         if t.startswith(w):
#             kwic[w].append(t)

In [70]:
# kwic = {term: FreqDist(kwic[term]) for term in kwic}

In [71]:
# for term in kw:
#     df_kw = DataFrame(kwic[term].items(), columns=['Concordance', "Fréquence (TF)"])
#     df_kw.sort_values(["Fréquence (TF)"], 
#         axis=0,
#         ascending=[False], 
#         inplace=True)

#     df_kw.insert(0, 'Mot-clé', term)
#     extrant = concat([extrant, df])


# extrant = extrant[extrant['Fréquence (TF)'] > 30] 

# # file_path = '../04-filtrage/output/'
# # file_path = path.join(file_path, acteur, tag)


# # extrant.to_csv(file_path + '_KWIC' +'.csv')

# extrant

### **Extraction de termes MeSH**

In [72]:
df['isMeSHTerm']= 'False' # On set à False puis on va changer pour True si on trouve le terme
df

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),isMeSHTerm
0,institut,NOM,568,-,-,108,False
1,psychiatrie,NOM,476,-,-,99,False
2,psychiatrie légale,NOM ADJ,416,-inf,1.0,94,False
3,candidature,NOM,285,-,-,71,False
4,philippe-pinel,NOM,233,-,-,94,False
...,...,...,...,...,...,...,...
417,programmes de traitement,NOM PRP NOM,16,200.173653,0.0,5,False
416,statut liste de rappel,NOM NOM PRP NOM,16,262.04235,0.0,10,False
415,temps complet permanent titre du poste,NOM ADJ ADJ NOM PRP:det NOM,16,190.608353,0.0,2,False
414,dotation,NOM,16,-,-,6,False


In [73]:
from nltk.tokenize import MWETokenizer
file_path = '../04-filtrage/MeSH/mesh-fr.txt'

with open (file_path, 'r', encoding='utf-8') as f:
    mesh = [tuple(tokenizer_re.tokenize(w)) for w in f.readlines()]
    tokenizer_mesh = MWETokenizer(mesh, separator= ' ')
    mesh = [tokenizer_mesh.tokenize(w)[0].lower() for w in mesh]
    mesh = [w for w in mesh if len(w.split()) > 1] # On ne retient que les termes complexes
    #mesh = [tuple(t.strip('.').lower().split()) for t in f.readlines()]

In [74]:
extr_mesh = tokenizer_mesh.tokenize(list_terms)

In [75]:
df

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),isMeSHTerm
0,institut,NOM,568,-,-,108,False
1,psychiatrie,NOM,476,-,-,99,False
2,psychiatrie légale,NOM ADJ,416,-inf,1.0,94,False
3,candidature,NOM,285,-,-,71,False
4,philippe-pinel,NOM,233,-,-,94,False
...,...,...,...,...,...,...,...
417,programmes de traitement,NOM PRP NOM,16,200.173653,0.0,5,False
416,statut liste de rappel,NOM NOM PRP NOM,16,262.04235,0.0,10,False
415,temps complet permanent titre du poste,NOM ADJ ADJ NOM PRP:det NOM,16,190.608353,0.0,2,False
414,dotation,NOM,16,-,-,6,False


In [76]:
for t in extr_mesh:
    if t in mesh:
        df.loc[df['Terme'] == t, 'isMeSHTerm'] = 'True'
df

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),isMeSHTerm
0,institut,NOM,568,-,-,108,False
1,psychiatrie,NOM,476,-,-,99,False
2,psychiatrie légale,NOM ADJ,416,-inf,1.0,94,True
3,candidature,NOM,285,-,-,71,False
4,philippe-pinel,NOM,233,-,-,94,False
...,...,...,...,...,...,...,...
417,programmes de traitement,NOM PRP NOM,16,200.173653,0.0,5,False
416,statut liste de rappel,NOM NOM PRP NOM,16,262.04235,0.0,10,False
415,temps complet permanent titre du poste,NOM ADJ ADJ NOM PRP:det NOM,16,190.608353,0.0,2,False
414,dotation,NOM,16,-,-,6,False


In [77]:
# Test - ne retenir uniquement que les termes qui sont des MeSH ? 
df[df['isMeSHTerm'] == 'True']

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),isMeSHTerm
2,psychiatrie légale,NOM ADJ,416,-inf,1.0,94,True
53,santé mentale,NOM ADJ,94,1111.572698,0.0,76,True
68,formation continue,NOM ADJ,81,779.537772,0.0,75,True
85,qualité de vie,NOM PRP NOM,72,981.302223,0.0,70,True
105,curriculum vitae,NOM ADJ,69,1113.514164,0.0,67,True
130,minorités ethniques,NOM ADJ,66,887.979725,0.0,66,True
221,soins infirmiers,NOM NOM,41,452.970751,0.0,15,True
250,congé de maladie,NOM PRP NOM,33,430.048217,0.0,59,True
291,services sociaux,NOM ADJ,27,209.028009,0.0,21,True
353,sécurité civile,NOM ADJ,22,271.19556,0.0,4,True


### **Extraction de termes existant dans la taxonomie**

In [78]:
df['isTaxoTerm']= 'False' # On set à False puis on va changer pour True si on trouve le terme
df

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),isMeSHTerm,isTaxoTerm
0,institut,NOM,568,-,-,108,False,False
1,psychiatrie,NOM,476,-,-,99,False,False
2,psychiatrie légale,NOM ADJ,416,-inf,1.0,94,True,False
3,candidature,NOM,285,-,-,71,False,False
4,philippe-pinel,NOM,233,-,-,94,False,False
...,...,...,...,...,...,...,...,...
417,programmes de traitement,NOM PRP NOM,16,200.173653,0.0,5,False,False
416,statut liste de rappel,NOM NOM PRP NOM,16,262.04235,0.0,10,False,False
415,temps complet permanent titre du poste,NOM ADJ ADJ NOM PRP:det NOM,16,190.608353,0.0,2,False,False
414,dotation,NOM,16,-,-,6,False,False


In [79]:
file_path = '../04-filtrage/default_taxo_labels.csv'

with open(file_path, 'r', encoding='utf-8') as f:
    default = read_csv(f, sep=';')
    labels = list(dict.fromkeys([str(t).strip().lower() for t in default['Label'].tolist()]))

labels

['1 on 1 support',
 '10 weeks and less ultrasound',
 '12-step facilitation therapy',
 '12-step programs',
 '18f pet bone scan',
 '18f bone pet',
 '18f bone scan',
 '1st generation antipsychotics',
 '2019 ncov testing',
 '2d doppler echocardiography',
 '2d echocardiography',
 '2d ultrasound',
 '2nd generation antipsychotics',
 '3d imaging',
 '3d sonography',
 '3d dental imaging',
 '3d echocardiography',
 '3d fetal ultrasound',
 '3d mammography',
 '3d prenatal ultrasound',
 '3d reconstruction',
 '46, xx disorder of sex development',
 '46,xy disorder of sex development',
 '4d echocardiography',
 '4d prenatal ultrasound',
 'a1c diabetes test',
 'a1c test',
 'abo + rh pnl bld',
 'abo and rh group in cord blood',
 'abpm device',
 'acl reconstruction surgery',
 'acth deficiency',
 'acth stimulation test',
 'adhd assessment',
 'adhd diagnostic',
 'adhd evaluation',
 'adhd testing',
 'adn foetal circulant',
 'aids',
 'aids nephropathy',
 'aids serodiagnosis',
 'aids-associated nephropathy',
 'a

In [80]:
for t in df['Terme'].tolist():
    if t in labels:
        df.loc[df['Terme'] == t, 'isTaxoTerm'] = 'True'
df

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),isMeSHTerm,isTaxoTerm
0,institut,NOM,568,-,-,108,False,False
1,psychiatrie,NOM,476,-,-,99,False,True
2,psychiatrie légale,NOM ADJ,416,-inf,1.0,94,True,False
3,candidature,NOM,285,-,-,71,False,False
4,philippe-pinel,NOM,233,-,-,94,False,False
...,...,...,...,...,...,...,...,...
417,programmes de traitement,NOM PRP NOM,16,200.173653,0.0,5,False,False
416,statut liste de rappel,NOM NOM PRP NOM,16,262.04235,0.0,10,False,False
415,temps complet permanent titre du poste,NOM ADJ ADJ NOM PRP:det NOM,16,190.608353,0.0,2,False,False
414,dotation,NOM,16,-,-,6,False,False


In [81]:
# Par exemple, on peut extraire les termes MeSH qui ne sont pas dans la taxonomie, mais qui se trouvent dans le corpus d'intérêt
df[df['isTaxoTerm'] == 'False'][df['isMeSHTerm'] == 'True'] #['Terme'].tolist()

C:\Users\p1115145\AppData\Local\Temp\ipykernel_292\4006263597.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df['isTaxoTerm'] == 'False'][df['isMeSHTerm'] == 'True'] #['Terme'].tolist()


,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),isMeSHTerm,isTaxoTerm
2,psychiatrie légale,NOM ADJ,416,-inf,1.0,94,True,False
68,formation continue,NOM ADJ,81,779.537772,0.0,75,True,False
85,qualité de vie,NOM PRP NOM,72,981.302223,0.0,70,True,False
105,curriculum vitae,NOM ADJ,69,1113.514164,0.0,67,True,False
130,minorités ethniques,NOM ADJ,66,887.979725,0.0,66,True,False
250,congé de maladie,NOM PRP NOM,33,430.048217,0.0,59,True,False
353,sécurité civile,NOM ADJ,22,271.19556,0.0,4,True,False


In [82]:
if lng == 'en':
    acteur = acteur + '_' + lng
if tag:
    output_path = path.join('../04-filtrage/output/', acteur + '_' + tag + '_significant-collocations.csv') 
else:
    output_path = path.join('../04-filtrage/output/', acteur + '_significant-collocations.csv') 

df['Corpus'] = acteur
df = df[["Corpus", "Terme", "Structure syntaxique",	"Fréquence (TF)", "Fréquence documentaire (DF)", "LLR", "isMeSHTerm", "isTaxoTerm" ]]
df.to_csv(output_path)
df

,Corpus,Terme,Structure syntaxique,Fréquence (TF),Fréquence documentaire (DF),LLR,isMeSHTerm,isTaxoTerm
0,pinel,institut,NOM,568,108,-,False,False
1,pinel,psychiatrie,NOM,476,99,-,False,True
2,pinel,psychiatrie légale,NOM ADJ,416,94,-inf,True,False
3,pinel,candidature,NOM,285,71,-,False,False
4,pinel,philippe-pinel,NOM,233,94,-,False,False
...,...,...,...,...,...,...,...,...
417,pinel,programmes de traitement,NOM PRP NOM,16,5,200.173653,False,False
416,pinel,statut liste de rappel,NOM NOM PRP NOM,16,10,262.04235,False,False
415,pinel,temps complet permanent titre du poste,NOM ADJ ADJ NOM PRP:det NOM,16,2,190.608353,False,False
414,pinel,dotation,NOM,16,6,-,False,False


### **Add MeSH ID & en/fr correspondance from extracted terms**

In [83]:
df['MeSHID'] = None

In [84]:
df

,Corpus,Terme,Structure syntaxique,Fréquence (TF),Fréquence documentaire (DF),LLR,isMeSHTerm,isTaxoTerm,MeSHID
0,pinel,institut,NOM,568,108,-,False,False,None
1,pinel,psychiatrie,NOM,476,99,-,False,True,None
2,pinel,psychiatrie légale,NOM ADJ,416,94,-inf,True,False,None
3,pinel,candidature,NOM,285,71,-,False,False,None
4,pinel,philippe-pinel,NOM,233,94,-,False,False,None
...,...,...,...,...,...,...,...,...,...
417,pinel,programmes de traitement,NOM PRP NOM,16,5,200.173653,False,False,None
416,pinel,statut liste de rappel,NOM NOM PRP NOM,16,10,262.04235,False,False,None
415,pinel,temps complet permanent titre du poste,NOM ADJ ADJ NOM PRP:det NOM,16,2,190.608353,False,False,None
414,pinel,dotation,NOM,16,6,-,False,False,None
